In [1]:
import os
import re
import pathlib
import time
import itertools
import glob
import datetime
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from IPython import display
from sklearn.model_selection import KFold

from deep_shadow import *
from utils import *

%load_ext autoreload
%autoreload 2

In [2]:
print(tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
True


In [3]:
cities = ['la','bos', 'dc', 'nyc','chi','sp', 'bue', 'joh', 'syd', 'tok', 'par', 'mex', 'sea', 'aus']
dates = ['winter', 'spring', 'summer']
zoom = 16
shadow_path = 'data/shadows/'
height_path = 'data/heights_new/'
checkpoint_name = 'evaluation_new'
checkpoint_path = 'training_checkpoints/%s'%(checkpoint_name)

BATCH_SIZE = 1
STEPS = 75000

if not os.path.exists('results/'):
    os.mkdir('results/')

In [ ]:
# Case 1: Train on K-1 Cities, Test on 1 City
# Training
for city in cities:
    train = cities[:]
    train.remove(city)
    
    train_dataset = get_tiles(height_path, shadow_path, train, dates, zoom, 350) # 400 * 13 * 3
    train_dataset = train_to_tensor(train_dataset, BATCH_SIZE)
    
    deep_shadow = DeepShadow(512,512)
    deep_shadow.fit(checkpoint_path+'/'+city, train_dataset, None, STEPS)
    
    tf.keras.backend.clear_session()

In [ ]:
# Case 1: Train on K-1 Cities, Test on 1 City
# Testing
for city in cities:
    test = [city]
    
    deep_shadow = DeepShadow(512, 512)
    deep_shadow.restore(checkpoint_path+'/'+city)
    
    test_dataset = get_tiles(height_path, shadow_path, test, dates, zoom, 1200)
    test_dataset = test_to_tensor(test_dataset, BATCH_SIZE)
    rmses, maes = get_metrics(test_dataset, deep_shadow.generator)
        
    print('\nCity %s rmse: %.2f, mae: %.2f'%(city, np.average(rmses), np.average(maes)))
    df = pd.DataFrame({'rmse': rmses, 'mae': maes})
    
    filepath = 'results/%s-%d/%s.csv'%(checkpoint_name, zoom, city)
    if not os.path.exists(os.path.dirname(filepath)):
        os.mkdir(os.path.dirname(filepath))
    
    df.to_csv(filepath, index=False, header=True)
    
    tf.keras.backend.clear_session()

In [ ]:
# Case 2: Train-Test on All Cities (5 Fold Cross Validation)
all_dataset = get_tiles(height_path, shadow_path, cities, dates, zoom, 400) 
all_dataset = np.array(all_dataset)

kf = KFold(n_splits=5, random_state=42, shuffle=True)

fold = 0
for train_index, test_index in kf.split(all_dataset):

    train_dataset = all_dataset[train_index]
    train_dataset = train_to_tensor(train_dataset, BATCH_SIZE)
    
    deep_shadow = DeepShadow(512, 512)
    deep_shadow.fit(checkpoint_path+'/fold_'+str(fold), train_dataset, None, STEPS)
    
    test_dataset = all_dataset[test_index]
    test_dataset = test_to_tensor(test_dataset, BATCH_SIZE)
    rmses, maes = get_metrics(test_dataset, deep_shadow.generator)
    
    print('\nFold: %d rmse: %.2f mae: %.2f '%(fold, np.average(rmses), np.average(maes)))
    df = pd.DataFrame({'rmse': rmses, 'mae': maes})
    
    filepath = 'results/%s-%d/fold_%d.csv'%(checkpoint_name, zoom, fold)
    if not os.path.exists(os.path.dirname(filepath)):
        os.mkdir(os.path.dirname(filepath))
    
    df.to_csv(filepath, index=False, header=True)
    
    fold+=1

In [7]:
# Case 3: Uniform Distribution of Latitudes (5 Fold Cross Validation)
# Training
# Paper
cities = ['aus', 'par', 'syd', 'sp', 'mex', 'la', 'chi']

all_dataset = get_tiles(height_path, shadow_path, cities, dates, zoom, 750) 
all_dataset = np.array(all_dataset)

kf = KFold(n_splits=5, random_state=42, shuffle=True)

fold = 0
for train_index, test_index in kf.split(all_dataset):
    
    train_dataset = all_dataset[train_index]
    train_dataset = train_to_tensor(train_dataset, BATCH_SIZE)
    
    deep_shadow = DeepShadow(512, 512)
    deep_shadow.fit(checkpoint_path+'/uniform_fold_'+str(fold), train_dataset, None, STEPS)
    
    test_dataset = all_dataset[test_index]
    test_dataset = test_to_tensor(test_dataset, BATCH_SIZE)
    rmses, maes, mses = get_metrics(test_dataset, deep_shadow.generator)
    
    print('\nFold: %d rmse: %.2f mae: %.2f mses: %.2f'%(fold, np.average(rmses), np.average(maes), np.average(mses)))
    df = pd.DataFrame({'rmse': rmses, 'mae': maes, 'mses': mses})
    
    filepath = 'results/%s-%d/uniform_fold_%d.csv'%(checkpoint_name, zoom, fold)
    if not os.path.exists(os.path.dirname(filepath)):
        os.mkdir(os.path.dirname(filepath))
    
    df.to_csv(filepath, index=False, header=True)
    
    fold+=1
    
    tf.keras.backend.clear_session()

Time taken for 1000 steps: 60.65 sec

Step: 74k
....................................................................................................
Fold: 4 rmse: 0.06 mae: 0.03 mses: 0.01


In [5]:
# Case 4: Uniform Distribution of Latitudes
# Training
# Paper
train_cities = ['aus', 'par', 'syd', 'sp', 'mex', 'la', 'chi']
test_cities = ['dc', 'nyc', 'joh', 'bue', 'bos', 'sea', 'tok']

train_dataset = get_tiles(height_path, shadow_path, train_cities, dates, zoom, 750)
train_dataset = train_to_tensor(train_dataset, BATCH_SIZE)

deep_shadow = DeepShadow(512,512)
deep_shadow.fit(checkpoint_path+'/uniform_cities', train_dataset, None, STEPS)

Time taken for 1000 steps: 63.88 sec

Step: 74k
....................................................................................................

In [6]:
# Case 4: Uniform Distribution of Latitudes
# Testing
# Paper
for city in test_cities:
    test = [city]
    
    deep_shadow = DeepShadow(512, 512)
    deep_shadow.restore(checkpoint_path+'/uniform_cities')
    
    test_dataset = get_tiles(height_path, shadow_path, test, dates, zoom, 750)
    test_dataset = test_to_tensor(test_dataset, BATCH_SIZE)
    rmses, maes, mses = get_metrics(test_dataset, deep_shadow.generator)
    
    print('\nCity %s rmse: %.2f mae: %.2f mses: %.2f'%(city, np.average(rmses), np.average(maes), np.average(mses)))
    df = pd.DataFrame({'rmse': rmses, 'mae': maes, 'mses': mses})
    
    filepath = 'results/%s-%d/uniform_%s.csv'%(checkpoint_name, zoom, city)
    if not os.path.exists(os.path.dirname(filepath)):
        os.mkdir(os.path.dirname(filepath))
    
    df.to_csv(filepath, index=False, header=True)
    
    tf.keras.backend.clear_session()


City dc rmse: 0.09 mae: 0.03 mses: 0.01

City nyc rmse: 0.09 mae: 0.04 mses: 0.01

City joh rmse: 0.03 mae: 0.01 mses: 0.00

City bue rmse: 0.05 mae: 0.02 mses: 0.00

City bos rmse: 0.08 mae: 0.03 mses: 0.01

City sea rmse: 0.07 mae: 0.04 mses: 0.01

City tok rmse: 0.09 mae: 0.05 mses: 0.01
